# Permafrost thaw rate [cm/yr]

In [1]:
import numpy as np
import h5py
import os, sys
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
import pandas as pd
sys.path.remove(sys.path[0])

sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/utils/')
import get_transect_data

sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata

In [2]:
path = "/Users/ajc/Core/simulations/barrow_icewedges/hydrologic_connection/ProjectionsK/"

In [3]:
#Simulations = [['CESM_C10','CESM_C11'], ['CESM_C20', 'CESM_C21'], ['CESM_C30', 'CESM_C31']]
Simulations = [['CESM_C10','CESM_C11'], ['CESM_C20', 'CESM_C21'], ['CESM_C30','CESM_C31','CESM_C32','CESM_C33'] ]# modified rain model
Sim_name = ['S1','S2','S3']

In [4]:
def getData(i,var_):
    Data = []
    Time = []
    for s in Simulations[i]:
        #print (i, s)
        infile = os.path.join(path,s)
        
        Tim1, Dat1 = get_transect_data.get2DMatrix(infile,varnames=[var_])
        a1 = len(Dat1[0,:,0,0])%73
        a2 = len(Dat1[0,:,0,0]) - a1
        
        Tim = Tim1[:a2]
        Dat = Dat1[:,:a2,:,:]
        Time.append(Tim)
        Data.append(Dat)
        #print ('HERE',a1,a2, Tim1.shape)
    d = dict()
    key_v = Sim_name[i] + '_' + var_
    key_t = Sim_name[i] + '_' + 'time'
    d[key_v] = Data
    d[key_t] = Time
    
    return d

In [5]:
#[nvar-1,i,j,zlen-depth_s:zlen-depth_e]
#d1 = getData(0)

In [6]:
from multiprocessing import Pool
from functools import partial
num_processors = 3
pool=Pool(processes = num_processors)

def call_data(var):
    global DataS1, DataS2, DataS3
    prod_x=partial(getData, var_=var)
    DataS1, DataS2, DataS3 = pool.map(prod_x,[i for i in range(len(Simulations))])
    Data = [DataS1, DataS2,DataS3]
    return Data

3 4801 45 85
3 5038 45 85
3 5261 45 85
3 862 45 85
3 864 45 85
3 1679 45 85
3 1971 45 85
3 584 45 85
3 4801 45 85
3 5038 45 85
3 5261 45 85
3 862 45 85
3 864 45 85
3 1679 45 85
3 1971 45 85
3 584 45 85


In [7]:
def getData_All(var_loc):
    DataSimOrig = dict()
    DataWC = call_data(var=var_loc)
    DataTemp = call_data(var='temperature')
    #DataVol = call_data(var='cell_volume')
    #print (DataTemp[0].keys())
    for i in range(len(Sim_name)): #number of simulations
        data_wc = []
        L = len(Simulations[i])
        Time = []
        Wcontent  = []
        for k in range(L): #k is for the number of directories for a simulation
            key = list(DataWC[i].keys())[0] #[0] ='water_content', [1] = 'time'
            key_temp = list(DataTemp[i].keys())[0]
            key_time = list(DataWC[i].keys())[1]
            #key_vol = list(DataVol[i].keys())[0]
            #print (i, key)
            data_time = np.array(DataWC[i][key_time][k])
            data_wc = np.array(DataWC[i][key][k])

            data_temp = np.array(DataTemp[i][key_temp][k])
            #data_vol = np.array(DataVol[i][key_vol][k])
            
            print (data_wc.shape,data_temp.shape,data_time.shape)
            nvar, cycles, xnum, znum = data_wc.shape

            
            sum1 = 0
            den = 916/0.018015
            for cyc in range(cycles):
                temp_index = np.where(data_temp[nvar-1,cyc,:,:] < 273.15)
                dat = data_wc[nvar-1,cyc,:,:][temp_index]
                #dat_v = data_vol[nvar-1,cyc,:,:][temp_index]
                
                sum1 =  np.sum(dat)/den
                
                Wcontent.append(sum1)
                Time.append(data_time[cyc])
                
                    
        DataSimOrig[Sim_name[i] + var_loc] = np.array(Wcontent)
        DataSimOrig[Sim_name[i] + '_time'] = np.array(Time)
    return DataSimOrig

In [8]:
DataSim = getData_All(var_loc='water_content')

(3, 5256, 45, 85) (3, 5256, 45, 85) (5256,)
(3, 1679, 45, 85) (3, 1679, 45, 85) (1679,)
(3, 5037, 45, 85) (3, 5037, 45, 85) (5037,)
(3, 1971, 45, 85) (3, 1971, 45, 85) (1971,)
(3, 4745, 45, 85) (3, 4745, 45, 85) (4745,)
(3, 803, 45, 85) (3, 803, 45, 85) (803,)
(3, 803, 45, 85) (3, 803, 45, 85) (803,)
(3, 584, 45, 85) (3, 584, 45, 85) (584,)


In [9]:
def get_moving_average1(d, m=5):
    d_avg = []
    len1 = len(d)
    for i in range(len1):
        if i < len1 - m:
            s1 = np.mean(d[i:i+m])
        else:
            s1 = np.mean(d[i:i+m])
        d_avg.append(s1)
    return d_avg
def get_moving_average(d, m=5):
    d_avg = []
    len1 = len(d)
    for i in range(len1):
        if i < m:
            s1 = np.mean(d[:i+1])
        else:
            s1= np.mean(d[i-m:i])

        d_avg.append(s1)
    return d_avg

In [ ]:
S1_ALT = [0.54,0.55,0.55,0.57,0.56,0.57,0.55,0.54,0.53,0.54,0.56,0.58,0.59,0.58,0.57,0.58,0.59,0.59,0.63,0.65,0.64,0.67,0.7,0.73,0.75,0.73,0.73,0.73,0.73,0.75,0.78,0.8,0.81,0.82,0.84,0.84,0.86,0.87,0.87,0.87,0.88,0.9,0.94,0.95,0.95,0.94,0.95,0.95,0.96,0.99,1.03,1.05,1.07,1.11,1.13,1.16,1.13,1.13,1.13,1.13,1.13,1.16,1.16,1.16,1.16,1.16,1.16,1.16,1.19,1.23,1.28,1.29,1.34,1.37,1.39,1.36,1.36,1.34,1.31,1.31,1.4,1.51,1.63,1.77,1.91,2.04,2.17,2.23,2.28,2.34,2.37,2.35,2.41,2.47,2.47]

S2_ALT = [0.57,0.58,0.58,0.6,0.59,0.6,0.59,0.58,0.57,0.58,0.59,0.61,0.62,0.61,0.6,0.61,0.62,0.62,0.65,0.68,0.67,0.7,0.73,0.76,0.79,0.78,0.79,0.8,0.8,0.8,0.83,0.84,0.85,0.85,0.86,0.86,0.87,0.87,0.87,0.87,0.88,0.9,0.94,0.95,0.95,0.94,0.94,0.94,0.95,0.98,1.02,1.05,1.07,1.11,1.13,1.16,1.13,1.13,1.13,1.13,1.13,1.16,1.16,1.16,1.16,1.16,1.16,1.16,1.19,1.22,1.25,1.26,1.31,1.34,1.39,1.39,1.41,1.39,1.36,1.34,1.41,1.53,1.68,1.85,2.01,2.14,2.26,2.34,2.4,2.47,2.53,2.55,2.58,2.64,2.64,2.64 ]

S3_ALT = [0.54,0.55,0.55,0.57,0.56,0.58,0.56,0.55,0.55,0.56,0.57,0.59,0.6,0.58,0.57,0.58,0.58,0.58,0.61,0.64,0.63,0.66,0.69,0.72,0.74,0.72,0.73,0.73,0.74,0.75,0.79,0.8,0.81,0.81,0.84,0.84,0.85,0.85,0.84,0.83,0.84,0.87,0.89,0.9,0.9,0.9,0.9,0.9,0.91,0.94,0.97,0.97,0.98,1.01,1.03,1.05,1.03,1.05,1.03,1.03,1.05,1.07,1.08,1.11,1.12,1.13,1.15,1.16,1.17,1.22,1.27,1.28,1.33,1.37,1.39,1.36,1.36,1.34,1.31,1.29,1.38,1.49,1.61,1.73,1.88,1.98,2.09,2.11,2.17,2.19,2.19,2.17,2.21,2.24,2.16]

ALT = [S1_ALT,S2_ALT,S3_ALT]

In [ ]:
%matplotlib qt
outfile='/Users/ajc/Documents/MyPapers/TroughBC/figures/Oct20/'
def Plot_PermafrostThawRate():
    fig, axs = plt.subplots(1,1, figsize=(5,3.5), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    
    plt.tight_layout()
    #axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    #Labels = [r'S$_{1}$', r'S$_{2}$', r'S$_{3}$']
    Labels = ['no-flow', 'free-flow', 'hybrid-flow']
    Colors = ['k','b','r','r']
    T = ['a','b','c','d']
    Keys = list(DataSim.keys())
    print (Keys)
    TD = [2.5, 2.4, 2]
    for i in range(3):
        #print (DataSim[Keys[2*i]])
        d = DataSim[Keys[2*i]]
        
        d = np.reshape(d,(-1,73))
        x_init = d[0][52] #52 is 260 day (Sept 17th)
        alt_depth = ALT[i]
        print (len(d), len(alt_depth))
        d = [ 100* (x_init - x[52])/((49.77 - al)*12.15) for x,al in zip(d,alt_depth)] # x_length = 12.15 m
        d_avg5yr = get_moving_average(d,m=5)
        
        t = np.reshape(DataSim[Keys[2*i+1]],(-1,73)).mean(axis=1)

        axs.plot(2006 + t, d_avg5yr,color=Colors[i],label=Labels[i])
        #axs[i].text(2006,3.7,'(%s)'%T[i],fontsize=12, fontweight='normal')
        axs.set_xlim((2007, 2100))
        axs.set_ylim((0, 7))
        #axs.set_yticks(np.arange(0,20.1,10))
        axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
        
        axs.set_ylabel(r'Permafrost thaw rate [cm yr$^{-1}$]',fontsize=12, fontweight='normal')
        axs.legend(loc='upper left',labelspacing=.1)
            
        #axs.set_ylim(135,138)
    print (outfile)
    plt.savefig(outfile+'permafrost_thawrate.pdf', bbox_inches='tight', dpi=500)
    plt.show()



In [ ]:
Plot_PermafrostThawRate()